In [ ]:
import os
import re
import importlib

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import data_utils

# importlib.reload(data_utils)

## Load and preprocess the data

In [ ]:
path = "/Users/jkamalu/Downloads/BosphorusDB"

In [ ]:
# Load the data and store in dictionary

lookup_table = data_utils.process_data(path)

In [ ]:
# Extract ndarray from dictionary and create index

lookup_table, landmarks, face_data = data_utils.convert_data(lookup_table)

In [ ]:
# Pad the face data to max width and height with z_min

w_max = max(x.shape[1] for x in face_data)
h_max = max(x.shape[0] for x in face_data)
z_min = min(np.min(x) for x in face_data)
print("max dim:", w_max, h_max)
print("min z:", z_min)

for i, f in enumerate(face_data):
    w_pad = (w_max - f.shape[1]) // 2
    h_pad = (h_max - f.shape[0]) // 2
    f_pad = ((h_pad, h_max - f.shape[0] - h_pad), (w_pad, w_max - f.shape[1] - w_pad), (0, 0))
    face_data[i] = np.pad(f, f_pad, mode="constant", constant_values=z_min)

In [ ]:
# Convert to numpy arrays

landmarks = np.array(landmarks)
face_data = np.array(face_data)
print("face_data shape:", face_data.shape)

In [ ]:
# Adjust and scale z channel

N, W, H, channels = face_data.shape

for i in range(face_data.shape[0]):
    lowest, low = np.sort(np.unique(face_data[i, :, :, 2]))[:2]
    face_data[i, :, :, 2][face_data[i, :, :, 2] == lowest] = low

scaler = MinMaxScaler()
scaler.fit(face_data[:, :, :, 2].reshape(-1, 1))
scaled_z = scaler.transform(face_data[:, :, :, 2].reshape(-1, 1)).reshape((N, W, H))
face_data[:, :, :, 2] = scaled_z

In [ ]:
# Visualize the face data

uid = "bs000"

fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(9, 6), subplot_kw={'xticks': [], 'yticks': []})
fig.subplots_adjust(left=0.03, right=0.97, hspace=0.3, wspace=0.05)

for ax, config in zip(axs.flat, lookup_table[uid]):
    idx = lookup_table[uid][config]
    ax.imshow(face_data[idx, :, :, 2])
    ax.set_title("_".join(config))

plt.tight_layout()
plt.show()